# Brief Métier : Prédiction de Churn Clients avec le Deep Learning

## 1. Contexte professionnel
Vous êtes Data Scientists dans une ESN (Entreprise de Services Numériques) qui accompagne les opérateurs télécoms dans la réduction de la perte d’abonnés.
Votre nouveau client, TelcoNova, souhaite anticiper les départs de ses clients (churn) afin d’orienter ses campagnes de rétention.
Il met à votre disposition un extrait anonymisé de sa base CRM (le jeu Telco Customer Churn, déjà pré-nettoyé en grande partie) et vous laisse 3 jours pour livrer un premier prototype de modèle de prédiction exploitable en production.
TelcoNova exige un livrable reproductible et facilement intégrable par ses équipes MLOps ; vous travaillerez en binôme, en suivant les bonnes pratiques Git / GitHub.



## 2. Votre mission

### 1. Explorer & préparer les données :

audit qualité, gestion des valeurs manquantes, etc.

In [2]:
import pandas as pd
import numpy as np

dataframe = pd.read_csv('../data/WA_Fn-UseC_-Telco-Customer-Churn.csv')
for index, column in enumerate ( dataframe.columns) :
    print(f"{index}: {column}, ", end='')
    num_values = dataframe[column].nunique()
    is_numeric = pd.api.types.is_numeric_dtype(dataframe[column])
    numeric_type = str(dataframe[column].dtype)
    print(f"{num_values}{(' '+numeric_type) if is_numeric else ''} values", end='')
    if num_values < 10 :
        print(f": " + ', '.join(str(x) for x in dataframe[column].unique())) 
    else :
        print(f".") 



0: customerID, 7043 values.
1: gender, 2 values: Female, Male
2: SeniorCitizen, 2 int64 values: 0, 1
3: Partner, 2 values: Yes, No
4: Dependents, 2 values: No, Yes
5: tenure, 73 int64 values.
6: PhoneService, 2 values: No, Yes
7: MultipleLines, 3 values: No phone service, No, Yes
8: InternetService, 3 values: DSL, Fiber optic, No
9: OnlineSecurity, 3 values: No, Yes, No internet service
10: OnlineBackup, 3 values: Yes, No, No internet service
11: DeviceProtection, 3 values: No, Yes, No internet service
12: TechSupport, 3 values: No, Yes, No internet service
13: StreamingTV, 3 values: No, Yes, No internet service
14: StreamingMovies, 3 values: No, Yes, No internet service
15: Contract, 3 values: Month-to-month, One year, Two year
16: PaperlessBilling, 2 values: Yes, No
17: PaymentMethod, 4 values: Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)
18: MonthlyCharges, 1585 float64 values.
19: TotalCharges, 6531 values.
20: Churn, 2 values: No, Yes


In [11]:
# exploration des colonnes à plus de 10 valeurs

def check_float(dataframe : pd.DataFrame, columnname : str) :
    print(f"{columnname}:")
    numericount =0
    for value in dataframe[columnname].unique() :
        try :
            numeric_value = float(value)
            numericount+=1
        except :
            print(f"  [{value}]")

    print(f"{numericount} values")
    print()

# 0: customerID, 7043 values => drop column
# 5: tenure, 73 int64 values => int64
# 18: MonthlyCharges, 1585 float64 values => float64
check_float(dataframe, 'MonthlyCharges')

# 19: TotalCharges, 6531 values : Exploration
check_float(dataframe, 'TotalCharges')


MonthlyCharges:
1585 values

TotalCharges:
  [ ]
6530 values



In [ ]:
from ..app.pipeline_functions import exclude_spaces

encodage des variables / normalisation / standardisation des variables


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer

preprocessor = make_pipeline(
    make_column_transformer(
        (make_pipeline(
                FunctionTransformer(convert_age_to_study_years, validate=False),
                StandardScaler()
            ), ['age']
        ),
        (StandardScaler(), ['income']
        ),
        (make_pipeline(
                FunctionTransformer(lambda x: x.squeeze(), validate=False),
                TfidfVectorizer()
            ), ['description']
        ),

        (OneHotEncoder(), ["gender"]), # 2 values: Female, Male
        (OneHotEncoder(), ["SeniorCitizen"]), #, 2 int64 values: 0, 1
        (OneHotEncoder(), ["Partner"]), #2 values: Yes, No
        (OneHotEncoder(),["Dependents"]), # 22 values: No, Yes
        (OneHotEncoder(),["tenure"]), # 73 int64 values.
        (OneHotEncoder(),["PhoneService"]), # 2 values: No, Yes
        (OneHotEncoder(),["MultipleLines"]), # 3 values: No phone service, No, Yes
        (OneHotEncoder(),["InternetService"]), # 3 values: DSL, Fiber optic, No
        (OneHotEncoder(),["OnlineSecurity"]), #  3 values: No, Yes, No internet service
        (OneHotEncoder(),["OnlineBackup"]), # 3 values: Yes, No, No internet service
        (OneHotEncoder(),["DeviceProtection"]), # 3 values: No, Yes, No internet service        
        (OneHotEncoder(),["TechSupport"]), # 3 values: No, Yes, No internet service
        (OneHotEncoder(),["StreamingTV"]), # 3 values: No, Yes, No internet service
        (OneHotEncoder(),["StreamingMovies"]), # 3 values: No, Yes, No internet service
        (OneHotEncoder(),["Contract"]), # 3 values: Month-to-month, One year, Two year
        (OneHotEncoder(),["PaperlessBilling"]), # 2 values: Yes, No
        (OneHotEncoder(),["PaymentMethod"]), # 4 values: Electronic check, Mailed check, Bank transfer (automatic), Credit card (automatic)
        (OneHotEncoder(),["MonthlyCharges"]), # 1585 float64 values.
        (OneHotEncoder(),["TotalCharges"]), # 6531 values.
        (OneHotEncoder(),["Churn"]), # 2 values: No, Yes)

    )
)



### 2. Élaborer un pipeline d’entraînement sous TensorFlow Keras ou PyTorch (au choix) :


architecture MLP pensée pour la tabulaire : ≥ 2 couches cachées,


fonction de perte adaptée à une classification binaire + gestion éventuelle du déséquilibre de classes.


callbacks (EarlyStopping, ModelCheckpoint, TensorBoard ou équivalent) pour suivre l’entraînement,


suivi des métriques : ROC-AUC, F1-score, Recall sur la classe Churn = Yes.




### 3. Évaluer, comparer, justifier :


fixer un baseline simple (régression logistique ou arbre de décision) pour mesurer le gain du réseau,


analyser les courbes d’apprentissage, la matrice de confusion, le ROC, les distributions des scores,


expliquer (au moins qualitativement) les variables les plus influentes.




### 4. Exporter le modèle + artefacts :


binaire du modèle (SavedModel ↔ TensorFlow / fichier .pt ↔ PyTorch),


scaler / encoder,


script ou notebook d’inférence qui charge les artefacts et prédit le churn sur de nouvelles données.


## 3. Fonctionnalités obligatoires (MVP)


<table style="width:75%">
  <tr>
    <th>Bloc</th>
    <th>Exigences clés</th>
  </tr>
  <tr>
    <td>Data Prep</td>
    <td>
        <ul>
            <li>Nettoyage & typage correct de toutes les colonnes</li>
            <li>Encodage systématique des catégorielle</li>
            <li>Split train / val / test (stratifié)</li>
        </ul>
    </td>
  </tr>
  <tr>
    <td>Modélisation</td>
    <td>
        <ul>
            <li>MLP implémenté from scratch sous TF /Keras ou PyTorch (pas d’API AutoML)</li>
            <li>Backprop + optimiseur (Adam ou SGD)</li>
            <li>Gestion du déséquilibre (pondération ou autre)</li>
        </ul>
    </td>
  </tr>
  <tr>
    <td>Suivi & répétabilité</td>
    <td>
      <ul>
        <li>TensorBoard ou PyTorch Lightning logger </li>
        <li>Seeds fixés + README détaillant la reproduction</li>
        <li>ModelCheckpoint pour restaurer le meilleur modèle</li>
      </ul>
    </td>
  </tr>
  <tr>
    <td>Évaluation</td>
    <td>
      <ul>
        <li>ROC-AUC ≥ 0 .80 OU F1 ≥ 0 .60</li>
        <li>Rapport comparant le réseau au baseline</li>
      </ul>
    </td>
  </tr>
  <tr>
    <td>Collaboration Git</td>
    <td>
      <ul>
        <li>1 branch = 1 feature</li>
        <li> Pull request systématique avec description</li>
      </ul> 
    </td>
  </tr>
</table>





## 4. Détails techniques & considérations
Imbalance : la classe Churn = Yes ≈ 26 %.


Pré-processing dans le même pipeline que l’inférence (pas de données “fuites” entre train et test).


Hyperparamètres : batch_size, learning_rate, nb_neurones par couche ; explorez‐les rapidement (Random Search / KerasTuner / Ray Tune) mais documentez votre stratégie.


Éthique & RGPD : aucune donnée personnellement identifiable ne doit sortir du cadre du projet.


## 5. Ressources utiles

Docs officielles :
 

TensorFlow /Keras : https://www.tensorflow.org/api_docs


PyTorch : https://pytorch.org/docs/


scikit-learn preprocessing : https://scikit-learn.org/stable/modules/preprocessing.html


Outils :


KerasTuner, Optuna (hyper-tuning)


Imbalanced-learn (rééchantillonnage)


Cours/vidéos :


DeepLearning.AI TensorFlow Developer (Coursera) – chapitres 2 & 3


PyTorch Lightning Crash Course (YouTube)


## 6. Livrables
Code source python propre et documenté


ROC-AUC ≥ 0 .80 ou F1 ≥ 0 .60


README clair, pas de données personnelles commitées


Présentation prête (10 min max et tous les membres du groupes doivent parler)



Groupe pour le brief

